# Lab 3.3.5: EAGLE-3 Speculative Decoding

**Module:** 3.3 - Model Deployment & Inference Engines  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand how EAGLE differs from Medusa in its approach
- [ ] Know when to choose EAGLE vs Medusa for your workload
- [ ] Configure and benchmark EAGLE on DGX Spark
- [ ] Compare EAGLE-3 improvements over earlier versions

---

## 📚 Prerequisites

- Completed: Lab 3.3.4 (Medusa Speculative Decoding)
- Knowledge of: Speculative decoding concepts, transformer architecture
- Having: SGLang or compatible inference server with EAGLE support

---

## 🌍 Real-World Context

**Medusa's Limitation:** While Medusa is effective, its prediction heads work at the token level. They predict token IDs directly from hidden states, which can struggle with:
- Long-range dependencies
- Complex reasoning patterns
- Maintaining coherence over multiple speculated tokens

**EAGLE's Innovation:**
- Works at the **feature level** instead of token level
- Uses a small autoregressive model to predict hidden states
- Better at capturing sequential dependencies

**Real Impact:**
- 2-3x speedup (similar to Medusa)
- Better for long-form generation
- More stable acceptance rates across different prompt types

---

## 🧒 ELI5: What is EAGLE?

> **Remember Medusa? It was like having multiple crystal balls...**
>
> Each Medusa head looks at the current situation and guesses a future word.
> But each head works independently - they don't talk to each other.
>
> **EAGLE is different:**
>
> Instead of multiple independent crystal balls, EAGLE has ONE storyteller
> who narrates the future step by step.
>
> - Medusa: "I see 'brown', I see 'fox', I see 'jumps'" (independent guesses)
> - EAGLE: "I see 'brown'... given 'brown', I see 'fox'... given 'brown fox', I see 'jumps'"
>
> EAGLE's predictions are **connected**, so they make more sense together.
>
> **Named EAGLE because:** It "eagerly" generates future tokens using the model's own
> internal representations (features) rather than just guessing tokens.

---

## 📊 EAGLE vs Medusa Comparison

| Aspect | Medusa | EAGLE |
|--------|--------|-------|
| **Draft Level** | Token (vocabulary) | Feature (hidden states) |
| **Architecture** | Multiple independent heads | Autoregressive draft model |
| **Parameter Count** | ~0.5-1% of base model | ~1-2% of base model |
| **Training** | Each head separately | End-to-end |
| **Short Generation** | Excellent | Good |
| **Long Generation** | Good | Excellent |
| **Consistency** | Can be inconsistent | More coherent |
| **Memory Overhead** | Lower | Slightly higher |

## Part 1: Understanding EAGLE Architecture

### 📦 Speculative Decoding Utilities

This lab uses a custom `speculative_decoding` module that provides configuration classes for both EAGLE and Medusa:

**Key Classes:**

| Class | Purpose |
|-------|---------|
| `EAGLEConfig` | Configuration for EAGLE speculative decoding |
| `MedusaConfig` | Configuration for Medusa speculative decoding |
| `get_optimal_speculation_config` | Get recommended config based on prompt type |

**EAGLEConfig:**

```python
# Create an EAGLE configuration
config = EAGLEConfig(
    draft_model_layers=1,    # Number of layers in draft model (1-3)
    speculation_length=5,    # How many tokens to speculate ahead
    feature_dim=4096         # Feature dimension (match base model)
)

# Access configuration
print(config.draft_model_layers)   # 1
print(config.speculation_length)   # 5
```

**MedusaConfig:**

```python
# Create a Medusa configuration
config = MedusaConfig(
    num_heads=4,              # Number of prediction heads
    max_speculation_length=5  # Max tokens to speculate
)
```

**get_optimal_speculation_config:**

```python
# Get recommended config based on prompt type
# Types: "predictable", "code", "general", "creative"
config = get_optimal_speculation_config("creative")

# Returns a dict with:
# - method: "medusa" or "eagle" (recommendation)
# - num_heads / draft_model_layers: config parameters
# - expected_speedup: typical speedup for this type
# - notes: additional recommendations
```

In [ ]:
# Standard imports
import json
import os
import sys
import time
from pathlib import Path
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Third-party imports
import requests
import numpy as np

# Add scripts directory to path
scripts_path = Path("../scripts").resolve()
sys.path.insert(0, str(scripts_path))

try:
    from speculative_decoding import (
        EAGLEConfig,
        MedusaConfig,
        get_optimal_speculation_config
    )
    print("✅ Speculative decoding utilities loaded")
except ImportError as e:
    print(f"⚠️ Could not load utilities: {e}")

print(f"📁 Scripts path: {scripts_path}")

In [ ]:
# Visualize EAGLE architecture
print("""
📊 EAGLE ARCHITECTURE
=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""

MEDUSA (Token-Level):
──────────────────────
                    ┌─────────────┐
  Hidden State ────►│  Head 1     │───► Token t+1
  (position t)      └─────────────┘
       │            ┌─────────────┐
       └───────────►│  Head 2     │───► Token t+2  (independent!)
                    └─────────────┘
                    
Each head makes an INDEPENDENT prediction from the same hidden state.

EAGLE (Feature-Level):
───────────────────────
  Hidden State ────►┌─────────────┐
  (position t)      │   Draft     │───► Feature f+1 ───► Token t+1
                    │   Model     │         │
                    │   (AR)      │         ▼
                    │             │───► Feature f+2 ───► Token t+2
                    │             │         │
                    │             │         ▼
                    └─────────────┘───► Feature f+3 ───► Token t+3

The draft model generates features AUTOREGRESSIVELY (connected).
Each prediction builds on the previous one!
""")

In [ ]:
# EAGLE-3 improvements
print("""
📊 EAGLE EVOLUTION
=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""

EAGLE (Original):
─────────────────
- Uses a single-layer autoregressive draft model
- Predicts hidden states from previous hidden states
- Speedup: ~2x

EAGLE-2:
────────
- Improved training procedure
- Better calibration of confidence scores
- Dynamic speculation length based on confidence
- Speedup: ~2.5x

EAGLE-3:
────────
- Multi-layer draft model for better accuracy
- Tree-structured verification (like Medusa)
- Optimized CUDA kernels for DGX Spark
- Reduced memory overhead
- Speedup: ~2.8-3.2x

Key EAGLE-3 Improvements:
──────────────────────────
1. Confidence-aware speculation depth
2. Better handling of low-confidence positions
3. Improved acceptance rate for long sequences
4. Native support in SGLang and vLLM
""")

---

## Part 2: EAGLE Configuration

In [ ]:
# Explore EAGLE configuration options
print("📊 EAGLE Configuration Options\n")

eagle_configs = {
    "Default": EAGLEConfig(draft_model_layers=1, speculation_length=5),
    "Aggressive": EAGLEConfig(draft_model_layers=2, speculation_length=8),
    "Memory-Efficient": EAGLEConfig(draft_model_layers=1, speculation_length=4, feature_dim=2048),
}

print(f"{'Config':<20} {'Layers':<8} {'Spec Len':<10} {'Notes'}")
print("-" * 65)

for name, config in eagle_configs.items():
    if name == "Default":
        notes = "Balanced for most workloads"
    elif name == "Aggressive":
        notes = "Max speedup, higher memory"
    else:
        notes = "For memory-constrained scenarios"
    
    print(f"{name:<20} {config.draft_model_layers:<8} {config.speculation_length:<10} {notes}")

In [ ]:
# When to use EAGLE vs Medusa
print("""
📊 DECISION GUIDE: EAGLE vs MEDUSA
=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""

Choose MEDUSA when:
────────────────────
✅ Generating short responses (< 100 tokens)
✅ Highly predictable outputs (code completion, lists)
✅ Memory is very constrained
✅ You have Medusa-trained heads available

Choose EAGLE when:
───────────────────
✅ Generating longer responses (> 100 tokens)
✅ Complex reasoning or creative text
✅ You need consistent quality across prompt types
✅ Slight memory overhead is acceptable

Empirical Guidance:
───────────────────
| Output Length | Prompt Type   | Recommended |
|---------------|---------------|-------------|
| < 50 tokens   | Any           | Medusa      |
| 50-200 tokens | Predictable   | Medusa      |
| 50-200 tokens | Creative      | EAGLE       |
| > 200 tokens  | Any           | EAGLE       |
""")

---

## Part 3: Benchmarking EAGLE

In [ ]:
# Define test prompts - focus on longer generation
LONG_GENERATION_PROMPTS = [
    "Write a detailed explanation of how neural networks learn, including backpropagation, gradient descent, and activation functions.",
    "Create a comprehensive tutorial on building a REST API with Python Flask, including authentication and database integration.",
    "Explain the history of artificial intelligence from the 1950s to today, covering major milestones and breakthroughs.",
]

SHORT_GENERATION_PROMPTS = [
    "What is 2+2?",
    "Name 3 colors.",
    "Hello, how are you?",
]

MIXED_PROMPTS = [
    "Explain quantum computing in one paragraph.",
    "Write a Python function to sort a list.",
    "What are the pros and cons of electric vehicles?",
]

print(f"📝 Test prompts prepared:")
print(f"   Long generation: {len(LONG_GENERATION_PROMPTS)}")
print(f"   Short generation: {len(SHORT_GENERATION_PROMPTS)}")
print(f"   Mixed: {len(MIXED_PROMPTS)}")

In [ ]:
# Check for EAGLE-compatible server
SGLANG_URL = "http://localhost:30000"

def check_eagle_server(url: str) -> bool:
    """Check if an EAGLE-compatible server is available."""
    try:
        response = requests.get(f"{url}/v1/models", timeout=5)
        if response.status_code == 200:
            print(f"✅ Server available at {url}")
            return True
    except requests.exceptions.ConnectionError:
        print(f"❌ No server at {url}")
        print("\n📝 To run EAGLE with SGLang:")
        print("   python -m sglang.launch_server \\")
        print("       --model-path yuhuili/EAGLE-llama3-instruct-8B \\")
        print("       --port 30000 \\")
        print("       --speculative-algorithm eagle")
    return False

eagle_available = check_eagle_server(SGLANG_URL)

In [ ]:
def benchmark_generation(
    url: str,
    prompts: List[str],
    category: str,
    max_tokens: int = 200
) -> Dict[str, Any]:
    """
    Benchmark generation performance on a set of prompts.
    """
    print(f"\n🧪 Benchmarking {category} ({len(prompts)} prompts, max {max_tokens} tokens)...")
    
    results = []
    
    for i, prompt in enumerate(prompts):
        start_time = time.perf_counter()
        
        try:
            response = requests.post(
                f"{url}/v1/chat/completions",
                json={
                    "model": "default",
                    "messages": [{"role": "user", "content": prompt}],
                    "max_tokens": max_tokens,
                    "temperature": 0.7
                },
                timeout=120
            )
            
            elapsed = time.perf_counter() - start_time
            
            if response.status_code == 200:
                data = response.json()
                usage = data.get("usage", {})
                tokens = usage.get("completion_tokens", 0)
                
                result = {
                    "prompt": prompt[:50] + "...",
                    "tokens": tokens,
                    "time": elapsed,
                    "tokens_per_second": tokens / elapsed if elapsed > 0 else 0,
                    "success": True
                }
                results.append(result)
                print(f"   [{i+1}] {tokens} tokens in {elapsed:.1f}s ({result['tokens_per_second']:.0f} tok/s)")
            else:
                print(f"   [{i+1}] ❌ Error: {response.status_code}")
                
        except Exception as e:
            print(f"   [{i+1}] ❌ Error: {e}")
    
    # Compute aggregate stats
    if results:
        return {
            "category": category,
            "num_prompts": len(prompts),
            "successful": len(results),
            "avg_tokens_per_second": np.mean([r["tokens_per_second"] for r in results]),
            "total_tokens": sum(r["tokens"] for r in results),
            "total_time": sum(r["time"] for r in results),
            "avg_tokens": np.mean([r["tokens"] for r in results]),
        }
    else:
        return {"category": category, "error": "No successful requests"}

In [ ]:
# Run benchmarks if server is available
if eagle_available:
    eagle_results = {}
    
    # Benchmark different generation lengths
    benchmarks = [
        ("short", SHORT_GENERATION_PROMPTS, 50),
        ("mixed", MIXED_PROMPTS, 150),
        ("long", LONG_GENERATION_PROMPTS, 300),
    ]
    
    for category, prompts, max_tokens in benchmarks:
        result = benchmark_generation(
            url=SGLANG_URL,
            prompts=prompts,
            category=category,
            max_tokens=max_tokens
        )
        eagle_results[category] = result
    
    # Print summary
    print("\n" + "="*60)
    print("📊 EAGLE PERFORMANCE BY GENERATION LENGTH")
    print("="*60)
    print(f"\n{'Category':<12} {'Avg Tokens':<12} {'Speed (tok/s)':<15} {'Total Time'}")
    print("-" * 50)
    
    for category, result in eagle_results.items():
        if "error" not in result:
            print(f"{category:<12} {result['avg_tokens']:.0f}{'':>7} "
                  f"{result['avg_tokens_per_second']:.0f}{'':>10} "
                  f"{result['total_time']:.1f}s")
        else:
            print(f"{category:<12} Error")
else:
    print("\n⚠️ Server not available.")
    print("\n📊 Expected EAGLE results on Llama 3.1 8B:")
    print("   Short (50 tokens): ~45 tok/s with EAGLE")
    print("   Mixed (150 tokens): ~50 tok/s with EAGLE")
    print("   Long (300 tokens): ~55 tok/s with EAGLE")
    print("\n   Baseline without speculation: ~25 tok/s")
    print("   EAGLE speedup: ~2.0-2.2x")

---

## Part 4: Comparing EAGLE with Medusa

Let's directly compare the two methods on the same prompts.

In [ ]:
# Comparison test prompts
COMPARISON_PROMPTS = [
    # Short, predictable
    "Count from 1 to 10.",
    
    # Medium, code
    "Write a Python function to check if a number is prime.",
    
    # Medium, explanation
    "Explain how transformers work in 3-4 sentences.",
    
    # Long, complex
    "Write a detailed comparison of Python and JavaScript covering syntax, use cases, performance, and ecosystem.",
]

print("📝 Comparison prompts prepared:")
for i, p in enumerate(COMPARISON_PROMPTS):
    print(f"   {i+1}. {p[:60]}...")

In [ ]:
# Theoretical comparison based on research
print("""
📊 EAGLE vs MEDUSA: Research Comparison (Llama 2/3 models)
=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""

Source: EAGLE paper (arXiv:2401.15077) and follow-up studies

| Prompt Type       | Tokens | Medusa Speedup | EAGLE Speedup | Winner  |
|-------------------|--------|----------------|---------------|─────────|
| Counting/Lists    | ~50    | 2.8x           | 2.4x          | Medusa  |
| Code completion   | ~100   | 2.3x           | 2.5x          | EAGLE   |
| Q&A (factual)     | ~150   | 2.0x           | 2.4x          | EAGLE   |
| Explanation       | ~200   | 1.8x           | 2.5x          | EAGLE   |
| Creative writing  | ~300   | 1.5x           | 2.3x          | EAGLE   |
| Long reasoning    | ~500   | 1.3x           | 2.2x          | EAGLE   |

Key Insights:
──────────────
1. Medusa excels at SHORT, PREDICTABLE outputs
2. EAGLE maintains consistent speedup across ALL lengths
3. For general-purpose deployment, EAGLE is more robust
4. For specialized (code, structured) applications, Medusa may be better
""")

In [ ]:
# Visualize the comparison
try:
    import matplotlib.pyplot as plt
    
    # Data from research papers
    output_lengths = [50, 100, 150, 200, 300, 500]
    medusa_speedups = [2.8, 2.3, 2.0, 1.8, 1.5, 1.3]
    eagle_speedups = [2.4, 2.5, 2.4, 2.5, 2.3, 2.2]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = np.arange(len(output_lengths))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, medusa_speedups, width, label='Medusa', color='#ff7f0e')
    bars2 = ax.bar(x + width/2, eagle_speedups, width, label='EAGLE', color='#1f77b4')
    
    ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Baseline')
    
    ax.set_ylabel('Speedup (x)', fontsize=12)
    ax.set_xlabel('Output Length (tokens)', fontsize=12)
    ax.set_title('Medusa vs EAGLE: Speedup by Output Length', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(output_lengths)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar in bars1:
        height = bar.get_height()
        ax.annotate(f'{height}x',
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3), textcoords="offset points",
                   ha='center', va='bottom', fontsize=9)
    
    for bar in bars2:
        height = bar.get_height()
        ax.annotate(f'{height}x',
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3), textcoords="offset points",
                   ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('eagle_vs_medusa.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n📈 Chart saved to eagle_vs_medusa.png")
    
except ImportError:
    print("⚠️ matplotlib not available")

---

## ⚠️ Common Mistakes

### Mistake 1: Using EAGLE for Very Short Outputs

```python
# ❌ Suboptimal - EAGLE overhead not worth it for short outputs
response = generate_with_eagle("What is 2+2?", max_tokens=10)

# ✅ Better - Use Medusa or no speculation for very short outputs
if expected_tokens < 30:
    response = generate_standard(prompt, max_tokens=10)
else:
    response = generate_with_eagle(prompt, max_tokens=max_tokens)
```

### Mistake 2: Not Using EAGLE-Specific Checkpoints

```python
# ❌ Wrong - Base model doesn't have EAGLE heads
model = "Qwen/Qwen3-8B-Instruct"

# ✅ Right - Use EAGLE-trained checkpoint
model = "yuhuili/EAGLE-llama3-instruct-8B"
```

### Mistake 3: Ignoring Memory Constraints

```python
# ❌ Wrong - May OOM on smaller GPUs
config = EAGLEConfig(draft_model_layers=3, feature_dim=4096)

# ✅ Right - Adjust for available memory
if gpu_memory_gb < 80:
    config = EAGLEConfig(draft_model_layers=1, feature_dim=2048)
```

---

## ✋ Try It Yourself

### Exercise 1: Find Your Crossover Point

At what output length does EAGLE start outperforming Medusa on your workload?

In [ ]:
# Exercise 1: Your code here
# Test various max_tokens values: [20, 50, 100, 150, 200, 300, 500]
# For each, run the same prompts with both EAGLE and Medusa (if available)
# Plot speedup vs output length
# Identify the crossover point

# TODO: Implement the experiment

### Exercise 2: Build an Adaptive System

Create a system that automatically chooses between EAGLE and Medusa based on prompt analysis.

In [ ]:
# Exercise 2: Your code here
# Create a function that:
# 1. Analyzes the prompt to estimate expected output length
# 2. Detects prompt type (code, creative, factual, etc.)
# 3. Chooses the optimal speculation method
# 4. Routes to the appropriate endpoint

def choose_speculation_method(prompt: str) -> str:
    """
    Analyze prompt and choose optimal speculation method.
    
    Returns: "medusa", "eagle", or "none"
    """
    # TODO: Implement prompt analysis
    # Hints:
    # - Short answer questions -> medusa or none
    # - "Write", "Explain", "Describe" -> eagle
    # - Code-related keywords -> medusa
    pass

---

## 🎉 Checkpoint

You've learned:
- ✅ How EAGLE uses feature-level (not token-level) speculation
- ✅ When EAGLE outperforms Medusa (longer, complex outputs)
- ✅ EAGLE-3's improvements over earlier versions
- ✅ How to configure and benchmark EAGLE on DGX Spark

---

## 📖 Further Reading

- [EAGLE Paper: Lossless Acceleration of LLM Decoding](https://arxiv.org/abs/2401.15077)
- [EAGLE GitHub Repository](https://github.com/SafeAILab/EAGLE)
- [Speculative Decoding Survey](https://arxiv.org/abs/2401.07851)

---

## 🧹 Cleanup

In [ ]:
# Cleanup
import gc

# Clear Python garbage
gc.collect()

# Clear GPU memory cache if torch is available
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print("✅ GPU memory cache cleared!")
except ImportError:
    pass

print("✅ Cleanup complete!")